# 11-11.mn- News Classification

- Train a general language model.
- Fine-tune model on your target corpus.
- Train a classifier.

### Installing and importing libraties

In [ ]:
!pip install -Uqq torchtext==0.8.0
!pip install -Uqq fastai --upgrade
!pip install -Uqq fastcore --upgrade
!pip install -Uqq pandas==1.1.0

     |████████████████████████████████| 7.0MB 4.1MB/s 
     |████████████████████████████████| 204kB 4.3MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 10.5MB 5.6MB/s 


In [ ]:
!pip install -Uqq gdown

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [ ]:
import fastai
import fastcore
print('Fast.ai version:', fastai.__version__)
print('Fastcore version:', fastcore.__version__)

Fast.ai version: 2.3.1
Fastcore version: 1.3.20


In [ ]:
from fastai.text.all import *
from fastai.callback.progress import CSVLogger
import pandas as pd
import numpy as np
print('Pandas version:', pd.__version__)
print('numpy version:', np.__version__)

Pandas version: 1.1.5
numpy version: 1.19.5


In [ ]:
import gdown

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/Deep Learning/11-11'

/content/drive/MyDrive/Deep Learning/11-11


In [ ]:
!nvidia-smi

Tue May 11 12:10:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Pretrained model

In [ ]:
os.mkdir('models')
url = 'https://drive.google.com/uc?id=12c9UQFa5pea1Ef02sdEF-T9yTFPYbwoa'
output = 'models/mn_large_news20.pth'
gdown.download(url, output)

Downloading...
From: https://drive.google.com/uc?id=12c9UQFa5pea1Ef02sdEF-T9yTFPYbwoa
To: /content/drive/My Drive/Deep Learning/11-11/models/mn_large_news20.pth
177MB [00:01, 142MB/s]


'models/mn_large_news20.pth'

### Vocabulary

In [ ]:
url = 'https://drive.google.com/uc?id=1Go4hK1xuifDeFelpI-wAyxJh0i66gkn-'
output = 'models/mn_large_news20_vocab.pkl'
gdown.download(url, output)

Downloading...
From: https://drive.google.com/uc?id=1Go4hK1xuifDeFelpI-wAyxJh0i66gkn-
To: /content/drive/My Drive/Deep Learning/11-11/models/mn_large_news20_vocab.pkl
100%|██████████| 1.08M/1.08M [00:00<00:00, 86.4MB/s]


'models/mn_large_news20_vocab.pkl'

## Loading a dataset

In [ ]:
df = pd.read_csv('1111_dataset.csv')
df.head()

,agency,content,created_at,source_text,status_text,type_text
0,Нийслэлийн Засаг даргын Тамгын газар,Дулааны тухай,2012-10-13T16:33:29.484371Z,Дуудлага,Хаагдсан,Гомдол
1,Ерөнхий сайд,Ерөнхий сайдтай уулзах. Жолоочийн эрх ашиг алдагдаж байна.,2012-10-13T16:33:29.492197Z,Биечлэн,Хаагдсан,Санал хүсэлт
2,"Хүнс, хөдөө аж ахуй, хөнгөн үйлдвэрийн яам",Нефтийн үйлдвэр байгуулах тухай.,2012-10-13T16:33:29.495729Z,Дуудлага,Хаагдсан,Санал хүсэлт
3,Цагдаагийн ерөнхий газар,Жолооны үнэмлэх яагаад хэвлэгдэхгүй байна ?,2012-10-13T16:33:29.499487Z,Дуудлага,Хаагдсан,Санал хүсэлт
4,Эрүүл мэндийн яам,БГД Гэмтэл-н эмнэлгийн гадуурх хашааг нураах тухай,2012-10-13T16:33:29.503063Z,Дуудлага,Хаагдсан,Гомдол


In [ ]:
np.random.seed(42)
df['is_valid'] = np.random.choice([0,1], size=len(df), p=[0.9, 0.1])

In [ ]:
df.head()

,agency,content,created_at,source_text,status_text,type_text,is_valid
0,Нийслэлийн Засаг даргын Тамгын газар,Дулааны тухай,2012-10-13T16:33:29.484371Z,Дуудлага,Хаагдсан,Гомдол,0
1,Ерөнхий сайд,Ерөнхий сайдтай уулзах. Жолоочийн эрх ашиг алдагдаж байна.,2012-10-13T16:33:29.492197Z,Биечлэн,Хаагдсан,Санал хүсэлт,1
2,"Хүнс, хөдөө аж ахуй, хөнгөн үйлдвэрийн яам",Нефтийн үйлдвэр байгуулах тухай.,2012-10-13T16:33:29.495729Z,Дуудлага,Хаагдсан,Санал хүсэлт,0
3,Цагдаагийн ерөнхий газар,Жолооны үнэмлэх яагаад хэвлэгдэхгүй байна ?,2012-10-13T16:33:29.499487Z,Дуудлага,Хаагдсан,Санал хүсэлт,0
4,Эрүүл мэндийн яам,БГД Гэмтэл-н эмнэлгийн гадуурх хашааг нураах тухай,2012-10-13T16:33:29.503063Z,Дуудлага,Хаагдсан,Гомдол,0


In [ ]:
df.rename(columns = {'content' : 'text', 'type_text' : 'label'}, inplace = True)
df.head()

,agency,text,created_at,source_text,status_text,label,is_valid
0,Нийслэлийн Засаг даргын Тамгын газар,Дулааны тухай,2012-10-13T16:33:29.484371Z,Дуудлага,Хаагдсан,Гомдол,0
1,Ерөнхий сайд,Ерөнхий сайдтай уулзах. Жолоочийн эрх ашиг алдагдаж байна.,2012-10-13T16:33:29.492197Z,Биечлэн,Хаагдсан,Санал хүсэлт,1
2,"Хүнс, хөдөө аж ахуй, хөнгөн үйлдвэрийн яам",Нефтийн үйлдвэр байгуулах тухай.,2012-10-13T16:33:29.495729Z,Дуудлага,Хаагдсан,Санал хүсэлт,0
3,Цагдаагийн ерөнхий газар,Жолооны үнэмлэх яагаад хэвлэгдэхгүй байна ?,2012-10-13T16:33:29.499487Z,Дуудлага,Хаагдсан,Санал хүсэлт,0
4,Эрүүл мэндийн яам,БГД Гэмтэл-н эмнэлгийн гадуурх хашааг нураах тухай,2012-10-13T16:33:29.503063Z,Дуудлага,Хаагдсан,Гомдол,0


In [ ]:
df = df.drop(columns = ['agency','created_at','status_text','source_text'])
df.head()

,text,label,is_valid
0,Дулааны тухай,Гомдол,0
1,Ерөнхий сайдтай уулзах. Жолоочийн эрх ашиг алдагдаж байна.,Санал хүсэлт,1
2,Нефтийн үйлдвэр байгуулах тухай.,Санал хүсэлт,0
3,Жолооны үнэмлэх яагаад хэвлэгдэхгүй байна ?,Санал хүсэлт,0
4,БГД Гэмтэл-н эмнэлгийн гадуурх хашааг нураах тухай,Гомдол,0


### Fine Tuning

In [ ]:
dblocks = DataBlock(blocks=(TextBlock.from_df('text', is_lm=True)),
                    get_x=ColReader('text'), 
                    splitter=ColSplitter())

In [ ]:
dls = dblocks.dataloaders(df, bs=128, seq_len=80)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
lm_fns = ['mn_large_news20', 'mn_large_news20_vocab']

In [ ]:
learn = language_model_learner(dls, AWD_LSTM, drop_mult=0.5, pretrained=True, pretrained_fnames=lm_fns, 
                               metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
notebook_path = Path('').absolute()
learn.path = notebook_path